# YOLOv7: Trainable bag-of-freebies sets new state-of-the-art for real-time object detectors  
- paper review

## Introduction  
**What is bag-of-freebies?**  
- inference시에 추가적인 computation cost 없이 모델의 성능을 향상시키기 위한 method  
- e.g. reparameterization, label assignment  
 
- 이 논문의 contrubution들을 요약하자면 다음과 같음  
    - 1. detection accuracy를 개선하면서 inference cost는 증가시키지 않는 trainable bag-of-freebies method을 디자인함   
    - 2. re-parameterized method가 original module을 어떻게 대체할 것인지, 그리고 different output layers에서 label assignment를 어떻게 다룰 것인지에 대해서 의논  
    - 3. 효율적으로 parameters와 computation을 활용할 수 있는 extend and compound scaling method를 제안  
    - 4. 우리가 제안한 method는 parameters 수를 40% 감소하고 computation cost를 50% 감소하는 효과를 얻었으며 SOTA보다 더 빠르고 정확함  

## Related work  
### Real-time object detectors  
- 이 논문에서는 self-supervised learning이나 knowledge distillation method를 사용하지 않음  
- 대신, 더 robust한 loss function, 더 효율적인 label assignment, 더 효율적인 training method를 위해 new trainable bag-of-freebies method를 제안함  

### Model re-parameterization  
- model re-parameterization은 inference phase에서 multiple modules를 하나로 합치며 이를 두 가지로 나눌 수 있음  
    - model-level re-parameterization  
        - 서로 다른 training data로 동일한 여러 모델을 훈련한 후, weights를 average 취하는 것  
        - 서로 다른 iteration에서 모델의 weights를 average 취하는 것  
    - module-level re-parameterization  
        - training 과정에서 여러 modules를 동일한 modules로 또는 다른 modules branch로 분할하고 inference phase에는 하나의 modules로 합치는 것  

- 그러나 이러한 re-parametrization module은 여러 architectures에 완벽히 적용할 수 있지는 않음  
- 그래서 새로운 re-parameterization module과 관련 strategies를 제안  

### Model scaling  
- 일반적으로 model scaling은 서로 다른 scaling factors를 이용하며 예를 들자면 input size, number of layer, number of channel, number of feature pyramid 등이 있다  
- DenseNet, VoVNet과 같은 concatenation-based model은 일부 layers에서 input width를 바꾸는 것을 찾음  
- 우리가 제안할 architecture 역시 concatenation-based이기 때문에 새로운 compound scaling method를 제안함  

## Architecture  
### Extended efficient layer aggregation networks    
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLOv7/figures/figure2.png?raw=true">  


- parameters의 수, computation 등을 고려해 효율적인 architecture를 설계할 것임  
- CSPVoVNet의 archiecture는 다양한 layer에서 다양한 features를 학습할 수 있도록 gradient path를 분석함  
- 어떻게 효율적인 network를 디자인할 것인가?를 던졌을 때 저자들은 다음과 같은 결론을 내림  
- shortest and longest gradient path를 controll함으로써 deeper network가 효율적으로 학습하고 수렴할 수 있음  
- 그래서 Extended-ELAN (E-ELAN)을 제안함  
- large scale ELAN은 computational stacks 수와 gradient path 길이에 상관없이 학습할 때 안정적으로 수렴한다  
- 그러나 이 수가 많아지면 architecture가 파괴돼 학습이 불안정할 가능성이 클 수 있음  
- 우리가 제안한 E-ELAN은 expand, shuffle, merge cardinality를 사용해 original gradient path를 파괴하지 않고 학습 능력을 지속적으로 향상시킬 수 있음  

### Model scaling for concatenation-based models    
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLOv7/figures/figure3.png?raw=true">  


- model scaling의 주된 목적은 model의 일부 contributes를 조정하고 서로 다른 inference speed 요구를 만족시키기 위해 다양한 scale의 모델을 생성하는 것임  
- 우리는 compound scaling method를 제안하며 이는 모델의 initial design과 최적의 structure를 유지할 수 있음  



## Trainable bag-of-freebies  
### Planned re-parameterized convolution  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLOv7/figures/figure4.png?raw=true">  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLOv7/figures/figure6.png?raw=true">  



- 우리는 RepConv에서 identity connection이 ResNet의 residual, DenseNet의 concatenation 등에 적용하면 그 성능이 저하된다는 것을 발견  
- 그래서 우리는 identity connection 없이 RepConv를 사용함  

### Coarse for auxiliary and fine for lead loss    
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLOv7/figures/figure5.png?raw=true">  


- network architecture의 여러 head 부분에서 auxiliary head를 추가해 loss를 계산함으로써 모델의 성능을 개선  
- 과거에는 deep network 학습에서 label assignment는 일잔적으로 ground truth를 refer하고 주어진 rule에 따라 hard label을 생성했음  
- 그러나 최근, prediction의 distribution이나 quality를 이용해 soft label로 label assignment하는 연구가 나타남  
- 여기서, 저자들은 soft labels을 이용한 label assigner mechanism을 제안  
- 이를 통해 soft label이 source data와 target data 사이의 correlation과 distribution을 대표할 수 있게 할 수 있음  

## Experiments  
### Experimental setup  
- pre-trained model을 사용하지 않음  
- 위에서 제안한 다양한 methods를 적용해 YOLOv7-X, YOLOv7-W6, YOLOv7-E6, YOLOv7-E6E 등의 모델을 빌드  
- YOLOv7-tiny에는 activation function으로 leaky ReLU를 사용하고 나머지는 SiLU를 사용  

### Comprison of SOTA    
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLOv7/figures/table1.png?raw=true">
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLOv7/figures/table2.png?raw=true">
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLOv7/figures/table3.png?raw=true">
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLOv7/figures/table4.png?raw=true">

- SOTA 모델들과 비교해본 결과, YOLOv7이 전체적으로 성능이 우수함을 보여주고 있음  


## Conclusions  
- 이 논문은 real-time object detector의 새로운 architecture와 이에 해당하는 model scaling을 제안함  
- 또한 re-parameterized module의 replacement problem과 dynamic label assignment의 allocation problem을 발견  
- 이들을 해결하기 위해 우리는 trainable bag-of-freebies method를 제안했으며 이는 detection의 accuracy를 개선시킴